<a href="https://colab.research.google.com/github/Tommy-somen/nutrition_facts_2020/blob/master/%E9%A3%9F%E5%93%81%E6%A4%9C%E7%B4%A2%EF%BC%86Excel%E3%81%AB%E3%81%BE%E3%81%A8%E3%82%81%E3%82%8B%E3%83%97%E3%83%AD%E3%82%B0%E3%83%A9%E3%83%A0(r1).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 食品標準成分表2020をもとにした食品検索＆Excelにまとめるプログラム
---
- Google DriveにGithubに含まれる3ファイル(これを含む)を「栄養計算」フォルダにインプットしてください。
- 使用ファイル：
  - 食品検索＆Excelにまとめるプログラム(r1).ipynb
  - 20201225-mxt_kagsei-mext_01110_012.xlsx
  -ヘッダー.xlsx
- 参考：https://fooddb.mext.go.jp/index.pl
---
- 変更履歴(2022/02/22):g計算ができるように変更しました。

In [ ]:
#Google Driveにアクセス
from google.colab import drive
drive.mount("/content/drive")

Mounted at /content/drive


In [ ]:
cd /content/drive/MyDrive/栄養計算

/content/drive/MyDrive/栄養計算


# 1. はじめに実行する2点
- 1.1 はじめに実行してください。：driveへのアクセス、モジュールや関数の準備をします。
- 1.2 食品成分表の取得：excelファイルの情報を取得します。

In [ ]:
#@title 1.1 モジュール、変数、関数の取得

#モジュールのインポート
import numpy as np
import pandas as pd
import openpyxl
import pprint
import os
import shutil

#表示数の設定
pd.set_option('display.max_columns', 100)
pd.set_option('display.max_colwidth', 100)

#利用する関数
#excelに2次元配列を追記する関数
def write_list_2d(sheet, l_2d, start_row, start_col):
    for y, row in enumerate(l_2d):
        for x, cell in enumerate(row):
            sheet.cell(row=start_row + y,
                       column=start_col + x,
                       value=l_2d[y][x])

#１度全て文字列に変換する関する
def cast_string(df,cols):
  for col in cols:
    df[col] = df[col].astype("string")

  return df

  # * を消す関数
def del_astrisk(df,cols):
  for col in cols:
    df[col] = df[col].replace("*","0")
  return df

#( )を消す関数
def del_brackets(df,cols):
  for col in cols:
    try:
      df[col] = df[col].str.replace('(', "").str.replace(')',"")
    except AttributeError:
      pass

  return df

# - を消す関数
def del_bar(df,cols):
  for col in cols:
    df[col] = df[col].replace("-","0")

  return df

# Tr を消す関数
def del_Tr(df,cols):
  for col in cols:
    df[col] = df[col].replace("Tr","0")

  return df

#float64に変換する関数
def convert_float(df,cols):
  for col in cols:
    df[col] = df[col].astype(float)

  return df 

#十字架みたいなのを消す関数
def del_cross(df,cols):
  for col in cols:
    df[col] = df[col].str.replace("†","")

  return df

#元データをクレンジングして修正後のdfを返す関数
def cleansing_df(original_df, cols, calcable_cols, ):
  
  tmp2 = cast_string(original_df,cols)
  tmp3 = del_brackets(tmp2,cols)
  tmp4 = del_bar(tmp3,cols)
  tmp5 = del_Tr(tmp4,cols)
  tmp6 = del_cross(tmp5,cols)
  tmp7 = del_astrisk(tmp6, calcable_cols)
  tmp8 = convert_float(tmp7, calcable_cols)
  
  return tmp8

#変数
cols = ['廃棄率(%)', 'エネルギー(kj)', 'カロリー(kcal)', '水分(g)',
       '(たんぱく質)アミノ酸組成によるたんぱく質(g)', 'たんぱく質(g)', '(脂質)脂肪酸のトリアシルグリセロール当量(g)',
       '(脂質)コレステロール(mg)', '(脂質)脂質(g)', '(炭水化物)利用可能炭水化物（単糖当量）(g)',
       '(炭水化物)利用可能炭水化物（単糖当量）(g)(備考)', '(炭水化物)利用可能炭水化物（質量計）(g)',
       '(炭水化物)差引き法による利用可能炭水化物　(g)', '(炭水化物)差引き法による利用可能炭水化物　(g)(備考)',
       '(炭水化物)食物繊維総量(g)', '(炭水化物)糖アルコール(g)', '(炭水化物)炭水化物(g)', '有機酸(g)',
       '灰分(g)', '(無機質) ナトリウム(mg)', '(無機質) カリウム(mg)', '(無機質) カルシウム(mg)',
       '(無機質) マグネシウム(mg)', '(無機質) リン(mg)', '(無機質) 鉄(mg)', '(無機質) 亜鉛(mg)',
       '(無機質) 銅(mg)', '(無機質) マンガン(mg)', '空白', '(無機質) ヨウ素(μg)', '(無機質) セレン(μg)',
       '(無機質) クロム(μg)', '(無機質) モリブデン(μg)', '(ビタミンA)レチノール(μg)',
       '(ビタミンA) α|カロテン(μg)', '(ビタミンA) β|カロテン(μg)', '(ビタミンA) β|クリプトキサンチン(μg)',
       '(ビタミンA) β|カロテン当量(μg)', '(ビタミンA) レチノール活性当量(μg)', 'ビタミンD(μg)',
       '(ビタミンE) α|トコフェロール(mg)', '(ビタミンE)  β|トコフェロール(mg)',
       '(ビタミンE) γ|トコフェロール(mg)', '(ビタミンE) δ|トコフェロール(mg)', 'ビタミンK(μg)',
       'ビタミンＢ１(mg)', 'ビタミンB２(mg)', 'ナイアシン(mg)', 'ナイアシン当量(mg)', 'ビタミンＢ６(mg)',
       'ビタミンＢ１２(mg)', '葉酸(μg)', 'パントテン酸(mg)', 'ビオチン(μg)', 'ビタミンC(mg)',
       'アルコール(g)', '食塩相当量(g)']

calcable_cols = ['廃棄率(%)', 'エネルギー(kj)', 'カロリー(kcal)', '水分(g)',
       '(たんぱく質)アミノ酸組成によるたんぱく質(g)', 'たんぱく質(g)', '(脂質)脂肪酸のトリアシルグリセロール当量(g)',
       '(脂質)コレステロール(mg)', '(脂質)脂質(g)', '(炭水化物)利用可能炭水化物（単糖当量）(g)','(炭水化物)利用可能炭水化物（質量計）(g)',
       '(炭水化物)差引き法による利用可能炭水化物　(g)',
       '(炭水化物)食物繊維総量(g)', '(炭水化物)糖アルコール(g)', '(炭水化物)炭水化物(g)', '有機酸(g)',
       '灰分(g)', '(無機質) ナトリウム(mg)', '(無機質) カリウム(mg)', '(無機質) カルシウム(mg)',
       '(無機質) マグネシウム(mg)', '(無機質) リン(mg)', '(無機質) 鉄(mg)', '(無機質) 亜鉛(mg)',
       '(無機質) 銅(mg)', '(無機質) マンガン(mg)', '(無機質) ヨウ素(μg)', '(無機質) セレン(μg)',
       '(無機質) クロム(μg)', '(無機質) モリブデン(μg)', '(ビタミンA)レチノール(μg)',
       '(ビタミンA) α|カロテン(μg)', '(ビタミンA) β|カロテン(μg)', '(ビタミンA) β|クリプトキサンチン(μg)',
       '(ビタミンA) β|カロテン当量(μg)', '(ビタミンA) レチノール活性当量(μg)', 'ビタミンD(μg)',
       '(ビタミンE) α|トコフェロール(mg)', '(ビタミンE)  β|トコフェロール(mg)',
       '(ビタミンE) γ|トコフェロール(mg)', '(ビタミンE) δ|トコフェロール(mg)', 'ビタミンK(μg)',
       'ビタミンＢ１(mg)', 'ビタミンB２(mg)', 'ナイアシン(mg)', 'ナイアシン当量(mg)', 'ビタミンＢ６(mg)',
       'ビタミンＢ１２(mg)', '葉酸(μg)', 'パントテン酸(mg)', 'ビオチン(μg)', 'ビタミンC(mg)',
       'アルコール(g)', '食塩相当量(g)']


In [ ]:
#@title 1.2  食品成分表の取得
df = pd.read_excel("20201225-mxt_kagsei-mext_01110_012.xlsx",
                     keep_default_na=False, # 空欄をNaで置き換えない
                     header=0, # headerは指定しない
                     )

#df = df.drop(columns=["Unnamed: 0"])
df["食品名"] = df["食品名"].replace("\u3000", " ")

df = cleansing_df(df ,cols, calcable_cols)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:41: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.


# 2. 食品情報の検索（キーワードを指定して検索&検索結果をexcel形式で出力）
- ○2.1  検索と検索結果の表示：キーワードを入力すると、それが含まれる食品情報が表示されます。
- ○2.2  上記検索結果をExcelで出力：検索結果をExcelで出力できます。(結果を反映する新規ファイル名を入力してください)


In [ ]:
#@title ○2.1  検索と検索結果の表示
keyword = "\u3044\u3082\u985E"#@param {type:"string"}


new_data = []
all_data = list(df["食品名"])

vals = []
for name in all_data:
  if keyword in name:
    vals.append(name)

result = df[df["食品名"].isin(vals)]
result

In [ ]:
#@title ○2.2  上記検索結果をExcelで出力

#ファイルコピー
input_name = "\u4E73_\u691C\u7D22\u7D50\u679C4"#@param{type:"string"}
filename = "./"+input_name+".xlsx"
src = './ヘッダー.xlsx'
copy = "./tmp.xlsx"
shutil.copyfile(src,copy)

#excelシートの内容を取得
wb = openpyxl.load_workbook('./tmp.xlsx')
sheet = wb['表全体']

#検索結果のリスト化         
L = result.values.tolist()

#シートへの追記
write_list_2d(sheet, L, 13, 1)

#renameにして上書き
wb.save(filename)

#tmpファイルの削除
os.remove("./tmp.xlsx")

# 3.  取得したい食品情報の追加
- ○3.1  初期化：最初もしくは食品情報の追加を初めからやり直したい場合に実行してください。
- ○3.2  追加する食品情報の入力：食品情報の正式名称を入力してください。(検索結果をコピペを推奨します)
  - 複数追加する場合は、初期化を実行せずに名称を入力してください。

In [ ]:
#@title ○3.1 初期化

add_info_list = []

In [ ]:
#@title ○3.2  追加する食品情報の入力(推奨：検索結果の名称をコピペ)
add_food_info = "\uFF1C\u3044\u3082\u985E\uFF1E\u3000\uFF08\u3084\u307E\u306E\u3044\u3082\u985E\uFF09\u3000\u3060\u3044\u3058\u3087\u3000\u584A\u6839\u3000\u751F"#@param{type:"string"}
gram_info = "200"#@param{type:"string"}

if add_food_info == "":
  pass

else:
  for item in add_info_list:
    if  add_food_info == item[0]:
      print(add_food_info,"は既に追加済みです")
      break
    
  add_info_list.append([add_food_info,float(gram_info)])
  print(add_food_info,"を","{}グラム分追加しました。".format(gram_info))


＜いも類＞　（やまのいも類）　だいじょ　塊根　生 を 200グラム分追加しました。


# (4. ) 追加した食品情報の取り消し

In [ ]:
#@title 4.1  取り消したい食品情報を以下から選択してください。
if len(add_info_list) >= 1:
  for item in add_info_list:
    item2 = item[0]
    print(item2)
else:
  print("追加された食品情報は0件です。")

＜調味料類＞　（ドレッシング類）　乳化液状ドレッシング　ごまドレッシング


In [ ]:
#@title 4.2 取り消したい食品情報を入力してください。
remove_info = "\uFF1C\u8ABF\u5473\u6599\u985E\uFF1E\u3000\uFF08\u30C9\u30EC\u30C3\u30B7\u30F3\u30B0\u985E\uFF09\u3000\u4E73\u5316\u6DB2\u72B6\u30C9\u30EC\u30C3\u30B7\u30F3\u30B0\u3000\u3054\u307E\u30C9\u30EC\u30C3\u30B7\u30F3\u30B0"#@param{type:"string"}

if len(add_info_list) <= 0:
  print("追加数が0件のた削除対象はありません。")
else:
  i = 0
  for item in add_info_list:
    if item[0] == remove_info:
      add_info_list.pop(i)
      break
    else:
      i += 1
  print(remove_info,"を削除しました。")
  print("現在追加されている食品情報は以下となります。")
  for item in add_info_list:
     print(item[0])

＜調味料類＞　（ドレッシング類）　乳化液状ドレッシング　ごまドレッシング を削除しました。
現在追加されている食品情報は以下となります。


# 5.  Excelファイルで出力

In [ ]:
#@title 5.1  追加された食品情報をまとめてExcelとして出力


#空のdf
new_df = df[df["食品名"]=="xxxxxxxxxx"]
new_df

#g計算
for item in add_info_list:
  part_df = df[df["食品名"] == item[0]]

  for col in tmp_cols:
    part_df[col] = part_df[col]*(float(item[1])/100)
    
  new_df = pd.concat([new_df,part_df],axis=0)

added_df = new_df

#ファイルコピー
input_name = ""#@param{type:"string"}
filename = "./"+input_name+".xlsx"
src = './ヘッダー.xlsx'
copy = "./tmp.xlsx"
shutil.copyfile(src,copy)

#excelシートの内容を取得
wb = openpyxl.load_workbook('./tmp.xlsx')
sheet = wb['表全体']

#詳細情報のリスト化
add_l = added_df.values.tolist()

#シートへの追記
write_list_2d(sheet, add_l, 13, 1)

#renameにして上書き
wb.save(filename)

#tmpファイルの削除
os.remove("./tmp.xlsx")


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  del sys.path[0]
